In [1]:
import torch as t
from torch import nn
from torch.utils.data import Dataset, DataLoader
import plotly.express as px
from IPython.display import display
import pandas as pd
import numpy as np
import copy
from fancy_einsum import einsum
from dataclasses import dataclass
from tqdm.notebook import tqdm_notebook

from einops import rearrange, reduce, repeat

import utils
import cnn_modules as cm
import transformer_modules as tm

In [2]:
def single_head_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of self-attention (see the "Self-Attention in Detail" section of the Illustrated Transformer).

    With this function, you can ignore masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [3]:
def single_head_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor) -> t.Tensor:
    '''
    Should return the results of masked self-attention.

    See "The Decoder Side" section of the Illustrated Transformer for an explanation of masking.

    Q: shape (batch, seq_len, embed_size)
    K: shape (batch, seq_len, embed_size)
    V: shape (batch, seq_len, embed_size)

    Return: shape (batch, seq_len, embed_size)
    '''
    batch_size, seq_len, embed_size = Q.shape
    scores = Q @ t.transpose(K, -2, -1)
    scores /= Q.shape[-1] ** 0.5
    
    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values
    scores = scores.masked_fill(mask==0, -1e9)
    
    scores = t.softmax(scores, dim=-1)
    Z = einsum('B Seq Score, B Seq Emb -> B Seq Emb', scores, V)
    return Z
    

In [4]:
def multihead_masked_attention(Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
    '''
    Implements multihead masked attention on the matrices Q, K and V.

    Q: shape (batch, seq, nheads*headsize)
    K: shape (batch, seq, nheads*headsize)
    V: shape (batch, seq, nheads*headsize)

    returns: shape (batch, seq, nheads*headsize)
    '''
    Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
    V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

    batch_size, seq_len, nheads, headsize = Q.shape
    scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
    scores /= Q.shape[-1] ** 0.5

    # create lower-left triangle of ones, including the diagonal
    mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
    # fill with close-to-neg-inf values where mask==0
    scores = scores.masked_fill(mask==0, -1e9)

    scores = t.softmax(scores, dim=-1)
    Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
    Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
    return Z

In [5]:
T = t.randn(2, 10, 4)

In [6]:
multihead_masked_attention(T, T, T, 2)

tensor([[[-0.4250, -1.2791,  0.3309, -0.0430],
         [ 1.0429, -0.3341, -1.0332, -1.1759],
         [ 1.5949,  0.4967,  0.9637,  0.3075],
         [ 0.3104, -0.3475, -1.0738,  0.3520],
         [-1.3645, -2.0715, -1.7436,  0.4082],
         [-0.8806, -0.8165,  0.1935, -0.9053],
         [ 1.4138,  0.1504,  1.2332,  0.8739],
         [ 0.6120,  1.0954,  0.0758,  2.7395],
         [ 0.4158,  0.6477, -0.7851,  1.7287],
         [-0.7782, -0.3896,  0.3513,  2.2130]],

        [[ 2.1577,  2.7396,  0.1303, -0.2417],
         [ 0.4947, -0.4466, -1.8226, -1.6052],
         [-0.4235,  0.0316,  0.4204, -0.0341],
         [ 0.6060,  1.9567,  0.6113,  0.4441],
         [ 0.1013, -0.5964,  1.6228,  1.2764],
         [-1.4939,  0.1601, -1.3533,  0.5968],
         [-0.7637,  0.7469, -1.5529, -1.0893],
         [ 2.6578, -1.8390,  1.0307, -0.6706],
         [-1.3052,  0.2678, -0.1824, -0.5362],
         [ 0.0493,  0.1421, -1.3328, -0.8817]]])

In [7]:
class MultiheadMaskedAttention(nn.Module):
    W_QKV: nn.Linear
    W_O: nn.Linear

    def __init__(self, hidden_size: int, num_heads: int):
        super().__init__()
        self.num_heads = num_heads
        self.query_size = int(hidden_size / num_heads)
        self.qkv = cm.Linear(hidden_size, 3*hidden_size)
        self.ff = cm.Linear(hidden_size, hidden_size)

    def multihead_masked_attention(self, Q: t.Tensor, K: t.Tensor, V: t.Tensor, num_heads: int):
        '''
        Implements multihead masked attention on the matrices Q, K and V.

        Q: shape (batch, seq, nheads*headsize)
        K: shape (batch, seq, nheads*headsize)
        V: shape (batch, seq, nheads*headsize)

        returns: shape (batch, seq, nheads*headsize)
        '''
        Q = rearrange(Q, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        K = rearrange(K, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)
        V = rearrange(V, 'B S (nheads headsize) -> B S nheads headsize', nheads = num_heads)

        batch_size, seq_len, nheads, headsize = Q.shape
        scores = einsum('B Qseq nheads headsize, B Kseq nheads headsize -> B nheads Qseq Kseq', Q, K)
        scores /= Q.shape[-1] ** 0.5

        # create lower-left triangle of ones, including the diagonal
        mask = t.tril(t.ones(seq_len, seq_len), diagonal=0)
        # fill with close-to-neg-inf values where mask==0
        scores = scores.masked_fill(mask==0, -1e9)

        scores = t.softmax(scores, dim=-1)
        Z = einsum('B nheads Qseq Kseq, B Qseq nheads headsize -> B Qseq nheads headsize', scores, V)
        Z = rearrange(Z, 'B Qseq nheads headsize -> B Qseq (nheads headsize)')
        return Z

    def forward(self, x: t.Tensor) -> t.Tensor:
        '''
        x: shape (batch, seq, hidden_size)

        Return: shape (batch, seq, hidden_size)
        '''
        out = self.qkv(x)
        Q, K, V = t.tensor_split(out, 3, dim=-1)

        Z = self.multihead_masked_attention(Q, K, V, self.num_heads)

        return self.ff(Z)

In [8]:
@dataclass(frozen=True)
class TransformerConfig:
    '''Constants used throughout your decoder-only transformer model.'''

    num_layers: int
    num_heads: int
    vocab_size: int
    hidden_size: int
    max_seq_len: int
    dropout: float = 0.1
    layer_norm_epsilon: float = 1e-05

In [9]:
config = TransformerConfig(
    num_layers=4, 
    num_heads=2, 
    vocab_size=500, 
    hidden_size=64,
    max_seq_len=100,
    dropout=0.1)

In [10]:
class MLP(nn.Module):

    def __init__(self, hidden_size, dropout):
        super().__init__()
        self.linear1 = cm.Linear(hidden_size, 4 * hidden_size)
        self.gelu = tm.GELU()
        self.linear2 = cm.Linear(4 * hidden_size, hidden_size)
        self.dropout = tm.Dropout(p=dropout)

    def forward(self, x):
        out = self.gelu(self.linear1(x))
        out = self.dropout(self.linear2(out))
        return out

In [11]:
class DecoderBlock(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.attn = tm.MultiheadMaskedAttention(config.hidden_size, config.num_heads)
        self.lnorm1 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)
        self.mlp = tm.MLP(config.hidden_size, config.dropout)
        self.lnorm2 = tm.LayerNorm(config.hidden_size, eps=config.layer_norm_epsilon)

    def forward(self, x: t.Tensor) -> t.Tensor:
        normed_attn = self.lnorm1(self.attn(x))
        out = normed_attn + x
        normed_mlp = self.lnorm2(self.mlp(out))
        out = normed_mlp + out
        return out


In [12]:
class DecoderOnlyTransformer(nn.Module):

    def __init__(self, config: TransformerConfig):
        super().__init__()
        self.emb = nn.Embedding(config.vocab_size, config.hidden_size)
        self.pos_enc = tm.PositionalEncoding(config.max_seq_len, config.hidden_size)
        self.dropout = tm.Dropout(p=config.dropout)

        decoders = [DecoderBlock(config) for l in range(config.num_layers)]
        self.decoders = nn.Sequential(*decoders)
        
        self.post_norm = tm.LayerNorm(config.hidden_size)

    def forward(self, x: t.Tensor) -> t.Tensor:
        embedding = self.emb(x.long())
        embedding = self.pos_enc(embedding)
        embedding = self.dropout(embedding)
        embedding = embedding.to(t.float32)

        out = self.decoders(embedding)
        out = self.post_norm(out)

        out = einsum("B S E, V E -> B S V", out, self.emb.weight)

        return out

## Testing Transformer

In [13]:
from torch.utils.data import Dataset

class ReverseNumberDataset(Dataset):
    def __init__(self, seq_len, total_size):
        self.seq_len = seq_len
        self.total_size = total_size

    def __len__(self):
        return self.total_size

    def __getitem__(self, idx):
        text = t.randint(0, 9, (self.seq_len, ))
        label = text.flip(dims=[0])
        sample = (text, label)
        return sample

In [14]:
num_ds = ReverseNumberDataset(6, 1_000_000)
num_ds[5]

(tensor([4, 1, 6, 1, 0, 8]), tensor([8, 0, 1, 6, 1, 4]))

In [15]:
trainloader = DataLoader(num_ds, batch_size=256, shuffle=True)

In [16]:
config = TransformerConfig(
    num_layers=2, 
    num_heads=4, 
    vocab_size=10, 
    hidden_size=128,
    max_seq_len=6,
    dropout=0.1)

In [18]:
from typing import Callable


epochs = 1
loss_fn = nn.CrossEntropyLoss()
batch_size = 256

MODEL_FILENAME = "./w1d2_transformer_digits.pt"
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")

def train_transformer(trainloader: DataLoader, epochs: int, loss_fn: Callable) -> list:
    '''
    Defines a Transformer from our custom modules, and trains it on the reversed digit dataset.
    '''

    model = DecoderOnlyTransformer(config).to(device).train()
    optimizer = t.optim.Adam(model.parameters())
    loss_list = []
    accuracy_list = []

    for epoch in range(epochs):

        progress_bar = tqdm_notebook(trainloader)
        for (x, y) in progress_bar:

            x = x.to(t.float32)
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            print(logits.shape)
            logits = rearrange(logits, 'B S V -> (B S) V')
            print(logits)
            print(y.shape)
            y = rearrange(y, 'B S -> (B S)')


            loss = loss_fn(logits, y)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss_list.append(loss.item())

            with t.inference_mode():
                model.eval()
                preds = model(t.tensor([[1,2,3,4,5,6]]))
                preds = preds.argmax(dim=-1)
                model.train()

            progress_bar.set_description(f"Epoch = {epoch}, Preds = {preds.squeeze()}, Loss = {loss.item():.4f}")

    print(f"Saving model to: {MODEL_FILENAME}")
    t.save(model, MODEL_FILENAME)
    return loss_list, accuracy_list

loss_list, accuracy_list = train_transformer(trainloader, epochs, loss_fn)

fig = px.line(y=loss_list, template="simple_white")
fig.update_layout(title="Cross entropy loss on number sequences", yaxis_range=[0, max(loss_list)])
fig.show()

  0%|          | 0/3907 [00:00<?, ?it/s]

torch.Size([256, 6, 10])
tensor([[ -9.0308,  -2.1465,   9.3617,  ...,  67.4852, -17.0629,  -8.9215],
        [  2.1030, -11.7464,   9.8696,  ...,  -3.8675,  -5.8900,   2.0800],
        [  2.1476,  64.9843,  -3.5425,  ...,  -5.8304, -17.9390,  12.3942],
        ...,
        [-18.0878,  -2.8202,   4.5129,  ...,  16.1145,   2.9951,   2.9400],
        [  2.0763,  60.8979,  -6.2914,  ...,  -2.5443,  -7.6473,   6.6032],
        [ -4.6435,  -4.8882,  -5.4931,  ...,  17.5741,  -1.0358,   6.5697]],
       grad_fn=<ReshapeAliasBackward0>)
torch.Size([256, 6])
torch.Size([256, 6, 10])
tensor([[ -7.8912,  -2.1922,   5.0423,  ...,   9.2020,   3.0151,   7.0556],
        [  4.3721,  -4.2033,  10.8873,  ...,   1.0335,  -2.2165,  -2.5082],
        [  4.4851,  -1.9013,  12.2206,  ...,   3.5903,  -3.9738,  -2.5249],
        ...,
        [  7.1125,  -3.9777,  -2.8725,  ...,  28.2551,  -1.7154,  -4.4774],
        [  2.0387,  -7.6894,   8.5238,  ...,  42.3995,  -2.7701, -15.4901],
        [ 12.4140,   0.229

KeyboardInterrupt: 